In [1]:
# Import dependencies
import pandas as pd
from pathlib import Path
import matplotlib
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf

In [2]:
# Create a basic cleaning function
def clean_data(file_path, ted_variables, services_values, reason_values):

    # Load .csv file into a DataFrame
    df = pd.read_csv(file_path)
    
    # Select columns
    teds_reduced_df = df[ted_variables]

    # Remove value 6 'Death' from Reason column 
    teds_reduced_df = teds_reduced_df[teds_reduced_df.REASON != 6]

    # Add sucessful column equal copied from REASON columnto 1 based on passed list reasons_values and the rest to 0
    teds_reduced_df['SUCCESSFUL'] = teds_reduced_df['REASON']

    # Change values in SUCCESSFUL column to 1 for passed list of reason_values chosen to indicate successful outcome
    for reason in reason_values:
        teds_reduced_df['SUCCESSFUL'] = teds_reduced_df['SUCCESSFUL'].replace({reason: 1}).astype(int) 
    
    # Change all other values in SUCCESSFUL column that aren't 1 to un sucessful 0.
    teds_reduced_df.loc[teds_reduced_df.SUCCESSFUL != 1, 'SUCCESSFUL'] = 0

    # Filter for AGES 18 and older.  Values > 2 based on codebook
    teds_clean = teds_reduced_df[teds_reduced_df.AGE > 2]
    
    # Take out all rows with value -9 (Missing/unknown/not collected/invalid) in any column
    teds_clean = teds_clean.replace({-9: np.nan}).dropna().astype(int)

    # Comnine race values 1,3,6,9 that are less than 1% to a new value of 10. Keep values 2, 4, 5, 7, 8 as is.
    races = [1,3,6,9]
    for race in races:
        teds_clean['RACE'] = teds_clean['RACE'].replace({race: 10}).astype(int) 

    # SERVICES column: select outpatient treatment, values 6 and 7,  Rhab values 2, 4, 5, 
    teds_clean = teds_clean[teds_clean["SERVICES"].isin(services_values)]

    # Return 
    return teds_clean

In [3]:
#Set file path for input dataset
file_path = Path('Resources/tedsd_puf_2019.csv')

# Select features to use for analysis.  Must include 'SERVICES and 'REASON' 
ted_variables = ['VET', 'REGION', 'FREQ_ATND_SELF_HELP', 'PSYPROB', 'DSMCRIT', 'ALCDRUG', 'PSOURCE', 'NOPRIOR', 'AGE',
                'RACE', 'GENDER', 'EDUC', 'MARSTAT', 'EMPLOY', 'LIVARAG', 'SERVICES', 'SUB1', 'SUB2','ROUTE1', 'FRSTUSE1', 'ALCFLG', 
                'COKEFLG', 'MARFLG', 'MTHAMFLG', 'OPSYNFLG', 'HERFLG', 'FREQ1', 'REASON']

# Select treatment services for analysis
services_values = [6,7] 

# Select values from REASON column to combine to a value of 1 for the target column SUCCESSFUL.  Can only be any combination of 1,4,7.
reason_values = [1]

# Call clean data funtion to get cleaned data frame for output to machine learning model code
teds_cleaned_df = clean_data(file_path, ted_variables, services_values, reason_values)
teds_cleaned_df.head()

,VET,REGION,FREQ_ATND_SELF_HELP,PSYPROB,DSMCRIT,ALCDRUG,PSOURCE,NOPRIOR,AGE,RACE,...,FRSTUSE1,ALCFLG,COKEFLG,MARFLG,MTHAMFLG,OPSYNFLG,HERFLG,FREQ1,REASON,SUCCESSFUL
0,2,4,1,1,19,1,1,0,4,10,...,5,1,0,0,0,0,0,2,1,1
1,2,4,1,1,4,3,1,0,5,10,...,3,1,0,1,0,0,0,2,3,0
3,2,4,4,1,4,3,1,1,5,10,...,3,1,0,1,0,0,1,1,1,1
4,2,4,1,1,4,1,1,1,11,8,...,7,1,0,0,0,0,0,1,3,0
5,2,4,1,1,19,3,1,0,8,10,...,2,1,0,0,0,0,0,2,3,0


In [4]:
# Split our preprocessed data into our features and target arrays

y = teds_cleaned_df["SUCCESSFUL"].values
X = teds_cleaned_df.drop(["SUCCESSFUL","REASON"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, stratify=y)

C:\Users\cdono\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


In [5]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [6]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 15
hidden_nodes_layer2 = 10





nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))



# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 15)                420       
                                                                 
 dense_1 (Dense)             (None, 10)                160       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 591
Trainable params: 591
Non-trainable params: 0
_________________________________________________________________


In [7]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs= 10)

Epoch 1/10
11627/11627 [==============================] - 7s 589us/step - loss: 0.4913 - accuracy: 0.7779
Epoch 2/10
11627/11627 [==============================] - 7s 589us/step - loss: 0.4779 - accuracy: 0.7809
Epoch 3/10
11627/11627 [==============================] - 6s 558us/step - loss: 0.4752 - accuracy: 0.7812
Epoch 4/10
11627/11627 [==============================] - 6s 558us/step - loss: 0.4735 - accuracy: 0.7821
Epoch 5/10
11627/11627 [==============================] - 6s 557us/step - loss: 0.4725 - accuracy: 0.7822
Epoch 6/10
11627/11627 [==============================] - 6s 555us/step - loss: 0.4718 - accuracy: 0.7826
Epoch 7/10
11627/11627 [==============================] - 6s 548us/step - loss: 0.4713 - accuracy: 0.7828
Epoch 8/10
11627/11627 [==============================] - 6s 548us/step - loss: 0.4710 - accuracy: 0.7828
Epoch 9/10
11627/11627 [==============================] - 6s 550us/step - loss: 0.4708 - accuracy: 0.7830
Epoch 10/10
11627/11627 [=====================

In [8]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

3876/3876 - 1s - loss: 0.4715 - accuracy: 0.7826 - 1s/epoch - 324us/step
Loss: 0.4715264141559601, Accuracy: 0.7825890779495239


In [9]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 20
hidden_nodes_layer2 = 15





nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))



# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 20)                560       
                                                                 
 dense_4 (Dense)             (None, 15)                315       
                                                                 
 dense_5 (Dense)             (None, 1)                 16        
                                                                 
Total params: 891
Trainable params: 891
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs= 10)

Epoch 1/10
11627/11627 [==============================] - 7s 554us/step - loss: 0.4867 - accuracy: 0.7786
Epoch 2/10
11627/11627 [==============================] - 6s 552us/step - loss: 0.4753 - accuracy: 0.7815
Epoch 3/10
11627/11627 [==============================] - 6s 553us/step - loss: 0.4727 - accuracy: 0.7824
Epoch 4/10
11627/11627 [==============================] - 6s 551us/step - loss: 0.4715 - accuracy: 0.7825
Epoch 5/10
11627/11627 [==============================] - 6s 552us/step - loss: 0.4708 - accuracy: 0.7832
Epoch 6/10
11627/11627 [==============================] - 6s 552us/step - loss: 0.4702 - accuracy: 0.7833
Epoch 7/10
11627/11627 [==============================] - 7s 561us/step - loss: 0.4699 - accuracy: 0.7833
Epoch 8/10
11627/11627 [==============================] - 6s 554us/step - loss: 0.4695 - accuracy: 0.7836
Epoch 9/10
11627/11627 [==============================] - 6s 550us/step - loss: 0.4692 - accuracy: 0.7843
Epoch 10/10
11627/11627 [=====================

In [11]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

3876/3876 - 1s - loss: 0.4701 - accuracy: 0.7830 - 1s/epoch - 314us/step
Loss: 0.4700591564178467, Accuracy: 0.78298419713974


In [12]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 20
hidden_nodes_layer2 = 15





nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))



# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 20)                560       
                                                                 
 dense_7 (Dense)             (None, 15)                315       
                                                                 
 dense_8 (Dense)             (None, 1)                 16        
                                                                 
Total params: 891
Trainable params: 891
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs= 20)

Epoch 1/20
11627/11627 [==============================] - 7s 553us/step - loss: 0.4871 - accuracy: 0.7768
Epoch 2/20
11627/11627 [==============================] - 7s 566us/step - loss: 0.4747 - accuracy: 0.7814
Epoch 3/20
11627/11627 [==============================] - 6s 548us/step - loss: 0.4725 - accuracy: 0.7820
Epoch 4/20
11627/11627 [==============================] - 6s 550us/step - loss: 0.4714 - accuracy: 0.7825
Epoch 5/20
11627/11627 [==============================] - 6s 549us/step - loss: 0.4706 - accuracy: 0.7829
Epoch 6/20
11627/11627 [==============================] - 6s 549us/step - loss: 0.4701 - accuracy: 0.7831
Epoch 7/20
11627/11627 [==============================] - 6s 557us/step - loss: 0.4697 - accuracy: 0.7832
Epoch 8/20
11627/11627 [==============================] - 7s 566us/step - loss: 0.4692 - accuracy: 0.7834
Epoch 9/20
11627/11627 [==============================] - 6s 554us/step - loss: 0.4688 - accuracy: 0.7833
Epoch 10/20
11627/11627 [=====================

In [14]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

3876/3876 - 1s - loss: 0.4687 - accuracy: 0.7838 - 1s/epoch - 326us/step
Loss: 0.46869319677352905, Accuracy: 0.783822774887085


In [15]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 25
hidden_nodes_layer2 = 20





nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))



# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 25)                700       
                                                                 
 dense_10 (Dense)            (None, 20)                520       
                                                                 
 dense_11 (Dense)            (None, 1)                 21        
                                                                 
Total params: 1,241
Trainable params: 1,241
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs= 20)

Epoch 1/20
11627/11627 [==============================] - 7s 543us/step - loss: 0.4879 - accuracy: 0.7795
Epoch 2/20
11627/11627 [==============================] - 6s 541us/step - loss: 0.4758 - accuracy: 0.7821
Epoch 3/20
11627/11627 [==============================] - 6s 548us/step - loss: 0.4721 - accuracy: 0.7830
Epoch 4/20
11627/11627 [==============================] - 7s 567us/step - loss: 0.4706 - accuracy: 0.7834
Epoch 5/20
11627/11627 [==============================] - 6s 544us/step - loss: 0.4697 - accuracy: 0.7834
Epoch 6/20
11627/11627 [==============================] - 6s 547us/step - loss: 0.4691 - accuracy: 0.7835
Epoch 7/20
11627/11627 [==============================] - 6s 546us/step - loss: 0.4686 - accuracy: 0.7839
Epoch 8/20
11627/11627 [==============================] - 6s 547us/step - loss: 0.4682 - accuracy: 0.7848
Epoch 9/20
11627/11627 [==============================] - 6s 554us/step - loss: 0.4678 - accuracy: 0.7841
Epoch 10/20
11627/11627 [=====================

In [17]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

3876/3876 - 1s - loss: 0.4681 - accuracy: 0.7841 - 1s/epoch - 320us/step
Loss: 0.4680996537208557, Accuracy: 0.7840647101402283


In [18]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 25
hidden_nodes_layer2 = 20





nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))



# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 25)                700       
                                                                 
 dense_13 (Dense)            (None, 20)                520       
                                                                 
 dense_14 (Dense)            (None, 1)                 21        
                                                                 
Total params: 1,241
Trainable params: 1,241
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs= 25)

Epoch 1/25
11627/11627 [==============================] - 7s 559us/step - loss: 0.4864 - accuracy: 0.7783
Epoch 2/25
11627/11627 [==============================] - 6s 551us/step - loss: 0.4739 - accuracy: 0.7816
Epoch 3/25
11627/11627 [==============================] - 6s 543us/step - loss: 0.4715 - accuracy: 0.7829
Epoch 4/25
11627/11627 [==============================] - 6s 545us/step - loss: 0.4702 - accuracy: 0.7831
Epoch 5/25
11627/11627 [==============================] - 6s 548us/step - loss: 0.4692 - accuracy: 0.7838
Epoch 6/25
11627/11627 [==============================] - 6s 546us/step - loss: 0.4687 - accuracy: 0.7844
Epoch 7/25
11627/11627 [==============================] - 6s 544us/step - loss: 0.4681 - accuracy: 0.7844
Epoch 8/25
11627/11627 [==============================] - 6s 542us/step - loss: 0.4676 - accuracy: 0.7849
Epoch 9/25
11627/11627 [==============================] - 6s 547us/step - loss: 0.4673 - accuracy: 0.7853
Epoch 10/25
11627/11627 [=====================

In [20]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

3876/3876 - 1s - loss: 0.4687 - accuracy: 0.7841 - 1s/epoch - 331us/step
Loss: 0.4686528146266937, Accuracy: 0.7841372489929199


In [21]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 25
hidden_nodes_layer2 = 20





nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))



# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 25)                700       
                                                                 
 dense_16 (Dense)            (None, 20)                520       
                                                                 
 dense_17 (Dense)            (None, 1)                 21        
                                                                 
Total params: 1,241
Trainable params: 1,241
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs= 35)

Epoch 1/35
11627/11627 [==============================] - 7s 545us/step - loss: 0.4862 - accuracy: 0.7793
Epoch 2/35
11627/11627 [==============================] - 6s 546us/step - loss: 0.4744 - accuracy: 0.7820
Epoch 3/35
11627/11627 [==============================] - 6s 558us/step - loss: 0.4719 - accuracy: 0.7828
Epoch 4/35
11627/11627 [==============================] - 6s 547us/step - loss: 0.4705 - accuracy: 0.7832
Epoch 5/35
11627/11627 [==============================] - 6s 549us/step - loss: 0.4695 - accuracy: 0.7837
Epoch 6/35
11627/11627 [==============================] - 6s 548us/step - loss: 0.4689 - accuracy: 0.7840
Epoch 7/35
11627/11627 [==============================] - 7s 560us/step - loss: 0.4682 - accuracy: 0.7845
Epoch 8/35
11627/11627 [==============================] - 7s 575us/step - loss: 0.4678 - accuracy: 0.7844
Epoch 9/35
11627/11627 [==============================] - 7s 566us/step - loss: 0.4673 - accuracy: 0.7845
Epoch 10/35
11627/11627 [=====================

In [23]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

3876/3876 - 1s - loss: 0.4672 - accuracy: 0.7845 - 1s/epoch - 325us/step
Loss: 0.46719518303871155, Accuracy: 0.7845243215560913


In [24]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 25
hidden_nodes_layer2 = 20





nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))



# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 25)                700       
                                                                 
 dense_19 (Dense)            (None, 20)                520       
                                                                 
 dense_20 (Dense)            (None, 1)                 21        
                                                                 
Total params: 1,241
Trainable params: 1,241
Non-trainable params: 0
_________________________________________________________________


In [25]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs= 50)

Epoch 1/50
11627/11627 [==============================] - 7s 557us/step - loss: 0.4845 - accuracy: 0.7785
Epoch 2/50
11627/11627 [==============================] - 6s 555us/step - loss: 0.4736 - accuracy: 0.7822
Epoch 3/50
11627/11627 [==============================] - 6s 549us/step - loss: 0.4709 - accuracy: 0.7831
Epoch 4/50
11627/11627 [==============================] - 6s 548us/step - loss: 0.4695 - accuracy: 0.7841
Epoch 5/50
11627/11627 [==============================] - 6s 545us/step - loss: 0.4686 - accuracy: 0.7841
Epoch 6/50
11627/11627 [==============================] - 6s 544us/step - loss: 0.4679 - accuracy: 0.7848
Epoch 7/50
11627/11627 [==============================] - 6s 546us/step - loss: 0.4670 - accuracy: 0.7849
Epoch 8/50
11627/11627 [==============================] - 6s 545us/step - loss: 0.4665 - accuracy: 0.7854
Epoch 9/50
11627/11627 [==============================] - 6s 545us/step - loss: 0.4661 - accuracy: 0.7854
Epoch 10/50
11627/11627 [=====================

In [26]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

3876/3876 - 1s - loss: 0.4667 - accuracy: 0.7860 - 1s/epoch - 320us/step
Loss: 0.46672362089157104, Accuracy: 0.7860161066055298


In [30]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 25
hidden_nodes_layer2 = 20
hidden_nodes_layer3 = 1





nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))
# third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))



# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_25 (Dense)            (None, 25)                700       
                                                                 
 dense_26 (Dense)            (None, 20)                520       
                                                                 
 dense_27 (Dense)            (None, 1)                 21        
                                                                 
 dense_28 (Dense)            (None, 1)                 2         
                                                                 
Total params: 1,243
Trainable params: 1,243
Non-trainable params: 0
_________________________________________________________________


In [31]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs= 50)

Epoch 1/50
11627/11627 [==============================] - 7s 574us/step - loss: 0.5022 - accuracy: 0.7780
Epoch 2/50
11627/11627 [==============================] - 7s 574us/step - loss: 0.4750 - accuracy: 0.7815
Epoch 3/50
11627/11627 [==============================] - 7s 574us/step - loss: 0.4718 - accuracy: 0.7830
Epoch 4/50
11627/11627 [==============================] - 7s 575us/step - loss: 0.4702 - accuracy: 0.7836
Epoch 5/50
11627/11627 [==============================] - 7s 576us/step - loss: 0.4690 - accuracy: 0.7843
Epoch 6/50
11627/11627 [==============================] - 7s 575us/step - loss: 0.4681 - accuracy: 0.7839
Epoch 7/50
11627/11627 [==============================] - 7s 575us/step - loss: 0.4675 - accuracy: 0.7847
Epoch 8/50
11627/11627 [==============================] - 7s 576us/step - loss: 0.4672 - accuracy: 0.7845
Epoch 9/50
11627/11627 [==============================] - 7s 575us/step - loss: 0.4665 - accuracy: 0.7850
Epoch 10/50
11627/11627 [=====================

In [32]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

3876/3876 - 1s - loss: 0.4665 - accuracy: 0.7849 - 1s/epoch - 331us/step
Loss: 0.46647191047668457, Accuracy: 0.7848710417747498


In [33]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 30
hidden_nodes_layer2 = 25






nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))




# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_29 (Dense)            (None, 30)                840       
                                                                 
 dense_30 (Dense)            (None, 25)                775       
                                                                 
 dense_31 (Dense)            (None, 1)                 26        
                                                                 
Total params: 1,641
Trainable params: 1,641
Non-trainable params: 0
_________________________________________________________________


In [34]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs= 50)

Epoch 1/50
11627/11627 [==============================] - 7s 545us/step - loss: 0.4834 - accuracy: 0.7795
Epoch 2/50
11627/11627 [==============================] - 6s 549us/step - loss: 0.4728 - accuracy: 0.7825
Epoch 3/50
11627/11627 [==============================] - 6s 550us/step - loss: 0.4705 - accuracy: 0.7836
Epoch 4/50
11627/11627 [==============================] - 6s 548us/step - loss: 0.4691 - accuracy: 0.7841
Epoch 5/50
11627/11627 [==============================] - 6s 549us/step - loss: 0.4681 - accuracy: 0.7843
Epoch 6/50
11627/11627 [==============================] - 6s 549us/step - loss: 0.4674 - accuracy: 0.7850
Epoch 7/50
11627/11627 [==============================] - 6s 549us/step - loss: 0.4666 - accuracy: 0.7852
Epoch 8/50
11627/11627 [==============================] - 6s 549us/step - loss: 0.4660 - accuracy: 0.7853
Epoch 9/50
11627/11627 [==============================] - 6s 546us/step - loss: 0.4657 - accuracy: 0.7853
Epoch 10/50
11627/11627 [=====================

In [35]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

3876/3876 - 1s - loss: 0.4674 - accuracy: 0.7842 - 1s/epoch - 331us/step
Loss: 0.4674375355243683, Accuracy: 0.7842018008232117


In [36]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 25
hidden_nodes_layer2 = 20






nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))




# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_32 (Dense)            (None, 25)                700       
                                                                 
 dense_33 (Dense)            (None, 20)                520       
                                                                 
 dense_34 (Dense)            (None, 1)                 21        
                                                                 
Total params: 1,241
Trainable params: 1,241
Non-trainable params: 0
_________________________________________________________________


In [37]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs= 100)

Epoch 1/100
11627/11627 [==============================] - ETA: 0s - loss: 0.4852 - accuracy: 0.77 - 7s 552us/step - loss: 0.4851 - accuracy: 0.7787
Epoch 2/100
11627/11627 [==============================] - 6s 559us/step - loss: 0.4744 - accuracy: 0.7821
Epoch 3/100
11627/11627 [==============================] - 6s 555us/step - loss: 0.4719 - accuracy: 0.7825
Epoch 4/100
11627/11627 [==============================] - 6s 556us/step - loss: 0.4705 - accuracy: 0.7831
Epoch 5/100
11627/11627 [==============================] - 7s 562us/step - loss: 0.4695 - accuracy: 0.7839
Epoch 6/100
11627/11627 [==============================] - 6s 554us/step - loss: 0.4687 - accuracy: 0.7845
Epoch 7/100
11627/11627 [==============================] - 6s 550us/step - loss: 0.4678 - accuracy: 0.7844
Epoch 8/100
11627/11627 [==============================] - 6s 549us/step - loss: 0.4674 - accuracy: 0.7849
Epoch 9/100
11627/11627 [==============================] - 6s 552us/step - loss: 0.4669 - accuracy: 0.

11627/11627 [==============================] - 7s 573us/step - loss: 0.4616 - accuracy: 0.7878
Epoch 77/100
11627/11627 [==============================] - 7s 571us/step - loss: 0.4617 - accuracy: 0.7877
Epoch 78/100
11627/11627 [==============================] - 7s 565us/step - loss: 0.4617 - accuracy: 0.7879
Epoch 79/100
11627/11627 [==============================] - 7s 567us/step - loss: 0.4617 - accuracy: 0.7876
Epoch 80/100
11627/11627 [==============================] - 7s 564us/step - loss: 0.4617 - accuracy: 0.7877
Epoch 81/100
11627/11627 [==============================] - 7s 585us/step - loss: 0.4617 - accuracy: 0.7877
Epoch 82/100
11627/11627 [==============================] - 7s 564us/step - loss: 0.4616 - accuracy: 0.7877
Epoch 83/100
11627/11627 [==============================] - 7s 591us/step - loss: 0.4617 - accuracy: 0.7877
Epoch 84/100
11627/11627 [==============================] - 8s 653us/step - loss: 0.4616 - accuracy: 0.7875
Epoch 85/100
11627/11627 [===============

In [38]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

3876/3876 - 1s - loss: 0.4667 - accuracy: 0.7843 - 1s/epoch - 327us/step
Loss: 0.46667930483818054, Accuracy: 0.7843065857887268


In [40]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 50
hidden_nodes_layer2 = 40






nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))




# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_38 (Dense)            (None, 50)                1400      
                                                                 
 dense_39 (Dense)            (None, 40)                2040      
                                                                 
 dense_40 (Dense)            (None, 1)                 41        
                                                                 
Total params: 3,481
Trainable params: 3,481
Non-trainable params: 0
_________________________________________________________________


In [41]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs= 50)

Epoch 1/50
11627/11627 [==============================] - 7s 564us/step - loss: 0.4830 - accuracy: 0.7797
Epoch 2/50
11627/11627 [==============================] - 7s 586us/step - loss: 0.4719 - accuracy: 0.7827
Epoch 3/50
11627/11627 [==============================] - 7s 562us/step - loss: 0.4691 - accuracy: 0.7841
Epoch 4/50
11627/11627 [==============================] - 7s 561us/step - loss: 0.4670 - accuracy: 0.7853
Epoch 5/50
11627/11627 [==============================] - 7s 560us/step - loss: 0.4655 - accuracy: 0.7860
Epoch 6/50
11627/11627 [==============================] - 7s 564us/step - loss: 0.4646 - accuracy: 0.7859
Epoch 7/50
11627/11627 [==============================] - 7s 579us/step - loss: 0.4636 - accuracy: 0.7867
Epoch 8/50
11627/11627 [==============================] - 7s 563us/step - loss: 0.4628 - accuracy: 0.7870
Epoch 9/50
11627/11627 [==============================] - 7s 563us/step - loss: 0.4622 - accuracy: 0.7874
Epoch 10/50
11627/11627 [=====================

In [42]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

3876/3876 - 1s - loss: 0.4657 - accuracy: 0.7854 - 1s/epoch - 319us/step
Loss: 0.46574681997299194, Accuracy: 0.7854194045066833


In [44]:
import sklearn
y_pred = nn.predict(X_test_scaled)
confusion_matrix = sklearn.metrics.confusion_matrix(y_test, np.rint(y_pred))
# confusion_matrix = tf.math.confusion_matrix(y_test, np.rint(y_pred)

print(confusion_matrix)

[[90006  4921]
 [21690  7397]]


In [45]:
# Create a DataFrame from the confusion matrix.
nn_cm_df = pd.DataFrame(
    confusion_matrix, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

nn_cm_df

,Predicted 0,Predicted 1
Actual 0,90006,4921
Actual 1,21690,7397


In [46]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 50
hidden_nodes_layer2 = 20






nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))




# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_41 (Dense)            (None, 50)                1400      
                                                                 
 dense_42 (Dense)            (None, 20)                1020      
                                                                 
 dense_43 (Dense)            (None, 1)                 21        
                                                                 
Total params: 2,441
Trainable params: 2,441
Non-trainable params: 0
_________________________________________________________________


In [47]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs= 50)

Epoch 1/50
11627/11627 [==============================] - 7s 561us/step - loss: 0.4818 - accuracy: 0.7800
Epoch 2/50
11627/11627 [==============================] - 7s 561us/step - loss: 0.4724 - accuracy: 0.7824
Epoch 3/50
11627/11627 [==============================] - 7s 564us/step - loss: 0.4697 - accuracy: 0.7833
Epoch 4/50
11627/11627 [==============================] - 7s 565us/step - loss: 0.4682 - accuracy: 0.7843
Epoch 5/50
11627/11627 [==============================] - 7s 567us/step - loss: 0.4670 - accuracy: 0.7848
Epoch 6/50
11627/11627 [==============================] - 7s 569us/step - loss: 0.4662 - accuracy: 0.7849
Epoch 7/50
11627/11627 [==============================] - 7s 563us/step - loss: 0.4652 - accuracy: 0.7857
Epoch 8/50
11627/11627 [==============================] - 7s 575us/step - loss: 0.4645 - accuracy: 0.7861
Epoch 9/50
11627/11627 [==============================] - 7s 579us/step - loss: 0.4641 - accuracy: 0.7863
Epoch 10/50
11627/11627 [=====================

In [48]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

3876/3876 - 1s - loss: 0.4655 - accuracy: 0.7854 - 1s/epoch - 339us/step
Loss: 0.4654717445373535, Accuracy: 0.7854436039924622
